In [11]:
%%capture
%load_ext autoreload
%autoreload 2
#Basic Imports
import os,sys
os.chdir("/home/asebaq/MSMatch")

from tqdm import tqdm,trange
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay
import torch
import pandas

from datasets.ssl_dataset import SSL_Dataset
from datasets.data_utils import get_data_loader
from utils import get_model_checkpoints
from utils import net_builder
from utils import clean_results_df

In [12]:
#Path to the runs to load
csv_folder = "/home/asebaq/MSMatch/cvs_folder" #Path where tmp csv files are stored
folder = "/home/asebaq/MSMatch/trained_models/ms/50/fixmatch/eurosat_ms/FixMatch_archefficientnet-b0_batch64_confidence0.95_lr0.03_uratio7_wd0.0005_wu1.0_seed0_numlabels50_optSGD" #Path to the runs to load 
sort_criterion = "numlabels" # Accepted net, numlabels
seed_wanted = 1 # Seed wanted (the others will be filtered)

## Initialize parameters

In [13]:
checkpoints, run_args = get_model_checkpoints(folder)
if os.name == 'nt':
       [print(_.split("\\")[1]) for _ in checkpoints];
else:
       [print(_.split("/")[1]) for _ in checkpoints];

home


## Run all models

In [14]:
results = []
for checkpoint, args in zip(checkpoints,run_args):
    print("------------ RUNNING ", checkpoint, " -----------------")
    print(args)
    args["batch_size"] = 256
    args["data_dir"] = "/home/asebaq/MSMatch/data/"
    args["use_train_model"] = False
    args["load_path"] = checkpoint
    if args["seed"] == seed_wanted:
        checkpoint_path = os.path.join(args["load_path"])
        checkpoint = torch.load(checkpoint_path,map_location='cuda:0')
        load_model = (checkpoint["train_model"] if args["use_train_model"] else checkpoint["eval_model"])
        _net_builder = net_builder(args["net"],False,{})
        _eval_dset = SSL_Dataset(name=args["dataset"], train=False, data_dir=args["data_dir"], seed=args["seed"])
        eval_dset = _eval_dset.get_dset()
        net = _net_builder(num_classes=_eval_dset.num_classes, in_channels=_eval_dset.num_channels)
        net.load_state_dict(load_model)
        if torch.cuda.is_available():
            net.cuda()
        net.eval()
    
        
        
    
        eval_loader = get_data_loader(eval_dset, args["batch_size"], num_workers=1)
        label_encoding = _eval_dset.label_encoding
        inv_transf = _eval_dset.inv_transform
    
        
        print("------------ PREDICTING TESTSET -----------------")
        
        images, labels, preds = [],[],[]
        with torch.no_grad():
            for image, target in tqdm(eval_loader):
                image = image.type(torch.FloatTensor).cuda()
                logit = net(image)
                for idx,img in enumerate(image):
                    images.append(inv_transf(img.transpose(0,2).cpu().numpy()).transpose(0,2).numpy())
                preds.append(logit.cpu().max(1)[1])
                labels.append(target)
        labels = torch.cat(labels).numpy()
        preds = torch.cat(preds).numpy()
        test_report = classification_report(labels, preds, target_names=label_encoding, output_dict=True)
        test_report["params"] = args
        results.append(test_report)

------------ RUNNING  /home/asebaq/MSMatch/trained_models/ms/50/fixmatch/eurosat_ms/FixMatch_archefficientnet-b0_batch64_confidence0.95_lr0.03_uratio7_wd0.0005_wu1.0_seed0_numlabels50_optSGD/model_best.pth  -----------------
{'dataset': 'eurosat_ms', 'net': 'efficientnet-b0', 'batch': 64, 'confidence': 0.95, 'lr': 0.03, 'uratio': 7, 'wd': 0.0005, 'wu': 1.0, 'seed': 0, 'numlabels': 50, 'opt': 'SGD', 'iterations': 567000}


In [15]:
import pandas as pd
big_df = pd.DataFrame()
pd.set_option('display.max_columns', None)
for result in results:
    params = result["params"]
    df = pd.DataFrame(result)
    df.drop(list(params.keys()),inplace=True)
    df.drop(["support","recall","precision"],inplace=True)
    for key,val in params.items():
        df[key] = val
    df = df.set_index("dataset")
    big_df = big_df.append(df)
print(big_df)
small_df = clean_results_df(big_df, folder,sort_criterion)
small_df.to_csv(csv_folder + "_test_results.csv")

Empty DataFrame
Columns: []
Index: []


IndexError: index 0 is out of bounds for axis 0 with size 0